In [21]:
import pandas as pd 
from google.cloud import bigquery

#https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=covid19_jhu_csse&page=dataset&project=majestic-lead-418020&ws=!1m4!1m3!3m2!1sbigquery-public-data!2scovid19_jhu_csse
#usada debido a que es un estudio realizado por una universidad de ciencia computacional e ingeniería y ya que nuestra carrera es de ingeniería lo vi útl (se esta utilizando la tabla casos confirmados, "confirmed_cases").
#https://ukhsa-dashboard.data.gov.uk
#para el caso de la API ya que el anterior era un estudio americano opte por ver una comparación en otro continente.


#dentro del parentesis no toma el proyect client = bigquery.Client(proyect='bigquery-public-data.covid19_jhu_csse')
client = bigquery.Client()#('bigquery-public-data.covid19_jhu_csse')

#el primer slot se le debe colocar lo que sigue en bigquery-public-data.
dataset_ref = client.dataset("covid19_open_data", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [22]:
query = """
        SELECT distinct date, new_confirmed
        FROM bigquery-public-data.covid19_open_data.covid19_open_data
        WHERE country_name = 'Chile'
        """

In [23]:
query_job = client.query(query)
chilean_cases = query_job.to_dataframe()
chilean_cases

,date,new_confirmed
0,2022-08-15,70
1,2021-01-18,96
2,2022-08-16,188
3,2022-01-28,965
4,2021-02-19,312
...,...,...
41219,2021-01-08,51
41220,2022-05-10,51
41221,2022-01-28,51
41222,2021-02-24,51


In [24]:
chilean_cases.groupby("date").agg("sum")

,new_confirmed
date,
2020-01-01,0
2020-01-02,0
2020-01-03,0
2020-01-04,0
2020-01-05,0
...,...
2022-09-13,3742
2022-09-14,0
2022-09-15,0


In [ ]:
chilean_cases = chilean_cases.dropna().sort_values('date').reset_index(drop=True)
chilean_cases

,date,new_confirmed
0,2020-01-01,0
1,2020-01-02,0
2,2020-01-03,0
3,2020-01-04,0
4,2020-01-05,0
...,...,...
40485,2022-09-13,120
40486,2022-09-13,203
40487,2022-09-13,62
40488,2022-09-13,17


In [ ]:
new_confirmed.set_index('date').plot()

NameError: name 'new_confirmed' is not defined